In [ ]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import models
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
import cv2

print(torch.__version__)
device = torch.device('cuda:0')

In [ ]:
class Weedread(Dataset):
    def __init__(self, name, transform=None):
        hf = h5py.File(name, 'r')
        self.input_images = np.array(hf.get('data'), np.uint8)
        self.target_labels = np.array(hf.get('labels')).astype(np.long)
        self.transform = transform
        hf.close()

    def __len__(self):
        return self.input_images.shape[0]

    def __getitem__(self, idx):
        images = self.input_images[idx]
        classes = self.target_labels[idx][1]
        family =  self.target_labels[idx][0]
        if self.transform is not None:
            images = self.transform(images)
        images = images
        
        return images, classes, family

In [ ]:
INPUT_CHANNEL = 3
BATCH_SIZE = 1
normalize = transforms.Compose([
    #transforms.ToPILImage(),
    #transforms.Resize((96,96)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

classes = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", 
           "14", "15", "16", "17", "18", "19", "20", "21"]

data_path = os.path.dirname(os.getcwd()) + "/data/weed/"
Train_data = Weedread(data_path + "train.h5", transform=normalize)
Test_data = Weedread(data_path + "val.h5", transform=normalize)

# Train_dataloader = DataLoader(dataset=Train_data,
#                               batch_size = BATCH_SIZE,
#                               shuffle=True)
Test_dataloader = DataLoader(dataset=Test_data,
                              batch_size = BATCH_SIZE,
                              shuffle=True)

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

class My_Model(nn.Module):
    def __init__(self, input_channel=1, num_class=21, num_family = 5):
        super(My_Model, self).__init__()
        model = models.resnet18(pretrained=True)
        self.model_ft = torch.nn.Sequential(*(list(model.children())[:-1]))
        set_parameter_requires_grad(self.model_ft, False)

        self.family_fc = nn.Linear(512, num_family)
        self.class_fc = nn.Linear(512, num_class)
        
    
    def forward(self, x):
        # Perform the usual forward pass
        x = self.model_ft(x)
        x = torch.flatten(x, 1)
        x_class = self.class_fc(x)
        x_family = self.family_fc(x)
        return F.softmax(x_class, dim=1), F.softmax(x_family, dim=1)

In [ ]:
from torchsummary import summary
train_images, _, _ = next(iter(Test_dataloader))

_model = My_Model(num_class=21, num_family = 5)
_model.to(device)


print(_model)
#summary(_model, input_size= train_images[0].size())

In [ ]:
normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
inv_normalize = transforms.Normalize([-0.485/0.229, -0.456/0.224, -0.406/0.225], [1/0.229, 1/0.224, 1/0.225])

def tensor_to_img(t):
    """Convert normalized tensor in Cuda to cv2 image"""
    unnormalized = inv_normalize(t)
    npimg = np.transpose(unnormalized.cpu().numpy(), (1, 2, 0))
    return npimg

def img_to_cuda_tensor(img):
    tr_img = np.transpose(img, (2, 0, 1))
    t = torch.from_numpy(tr_img)
    t = normalize(t.float())
    return t.to(device)
    

def imshow(img, title):
    """Custom function to display the image using matplotlib"""    
    npimg = tensor_to_img(img)
    #plot the numpy image
    plt.figure(figsize = (4, 4))
    plt.axis("off")
    plt.imshow(npimg)
    plt.title(title)
    plt.show()

In [ ]:
class SaveFeatures():
    """hook function at forward step"""
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = output
    def close(self):
        self.hook.remove()
        
class FilterVisualizer():
    """Visualize by maximize average activation"""
    def __init__(self, model):
        self.model = model
        self.model.eval()
        set_parameter_requires_grad(self.model, True)

    def visualize(self, sz, layer, filter, upscaling_steps=10, upscaling_factor=1.2, lr=0.1, opt_steps=20, blur=None):
        imarray = (np.random.random((sz, sz, 3)) * 20 + 128.)/255. #random tensor image
        
        activations = SaveFeatures(layer)  # register hook
        for i in range(upscaling_steps):  # scale the image up upscaling_steps times
            img = img_to_cuda_tensor(imarray)
            img_var = Variable(img[None], requires_grad=True)  # convert image to Variable that requires grad
            optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
            if i > upscaling_steps/2:
                opt_steps_ = int(opt_steps*1.3)
            else:
                opt_steps_ = opt_steps
            for n in range(opt_steps_):  # optimize pixel values for opt_steps times
                optimizer.zero_grad()
                self.model(img_var)
                loss = -1*activations.features[0, filter].mean()
                loss.backward()
                optimizer.step()
            img = tensor_to_img(img_var.data[0])
            self.output = img
            sz = int(upscaling_factor * sz)  # calculate new image size
            imarray = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
            if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
                
        activations.close()
        return np.clip(self.output, 0, 1)
        
    def most_activated(self, image, layer, limit_top=None):
        activations = SaveFeatures(layer)  # register hook
        img = img_to_cuda_tensor(image)
        self.model(Variable(img[0]))
        
        mean_act = [activations.features[0,i].mean().data.cpu().numpy()[0] for i in range(activations.features.shape[1])]
        activations.close()
        return mean_act

def reconstructions_single_layer(FV, layer,layer_name,filters,
                    init_size=56, upscaling_steps=12, upscaling_factor=1.2, 
                    opt_steps=20, blur=5, lr=1e-1,
                    n_cols=5, cell_size=4,save_fig=False):
    
    imgs = []
   
    for i in range(len(filters)):
        pic = FV.visualize(init_size,layer, filters[i], 
                                 upscaling_steps=upscaling_steps, 
                                 upscaling_factor=upscaling_factor, 
                                 opt_steps=opt_steps, blur=blur,
                                 lr=lr)
        imgs.append(pic)
        
    #plot feature maps
    n_rows = ((len(imgs))//n_cols)
    fig, axes = plt.subplots(n_rows,n_cols, figsize=(cell_size*n_cols,cell_size*n_rows))
    for i,ax in enumerate(axes.flat):
        ax.grid(False)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        if i>=len(filters):
            pass

        ax.set_title("fmap " + str(filters[i]))

        ax.imshow(imgs[i])
    fig.suptitle(layer_name, fontsize="x-large",y=1.0)
    plt.tight_layout()
        
    return imgs

In [ ]:
_model.load_state_dict(torch.load('epochs/ResNet18-base-line.pt'), strict=False)
FV = FilterVisualizer(_model)
pruned_model = nn.Sequential(*list(_model.model_ft.children()))
l = [module for module in pruned_model.modules() 
     if (type(module) != nn.Sequential and type(module) != models.resnet.BasicBlock) ]

print(len(l))


In [ ]:
print(l[0])
imgs = reconstructions_single_layer(FV, l[0], "", list(range(0, 50)), lr = 0.01, opt_steps= 20, upscaling_steps = 10)

In [ ]:
print(l[10])
imgs = reconstructions_single_layer(FV, l[10], "", list(range(0, 50)), lr = 0.01, opt_steps= 20, upscaling_steps = 10)

In [ ]:
print(l[20])
imgs = reconstructions_single_layer(FV, l[20], "", list(range(0, 50)), lr = 0.01, opt_steps= 20, upscaling_steps = 10)

In [ ]:
print(l[25])
imgs = reconstructions_single_layer(FV, l[25], "", list(range(0, 50)), lr = 0.01, opt_steps= 20, upscaling_steps = 10)

In [ ]:
print(l[30])
imgs = reconstructions_single_layer(FV, l[30], "", list(range(0, 50)), lr = 0.01, opt_steps= 20, upscaling_steps = 10)

In [ ]:
print(l[35])
imgs = reconstructions_single_layer(FV, l[35], "", list(range(0, 50)), lr = 0.01, opt_steps= 20, upscaling_steps = 10)